In [7]:
import pandas as pd

In [8]:
data = pd.read_csv("C:\\Users\\This  PC\\Documents\\Financial\\Financial_inclusion_dataset.csv")

data.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [10]:
column_explanation = pd.read_csv("C:\\Users\\This  PC\\Documents\\Financial\\VariableDefinitions (1).csv")

column_explanation.head(14)

,Variable Definitions,Unnamed: 1
0,country,Country interviewee is in.
1,year,Year survey was done in.
2,uniqueid,Unique identifier for each interviewee
3,location_type,"Type of location: Rural, Urban"
4,cellphone_access,"If interviewee has access to a cellphone: Yes, No"
5,household_size,Number of people living in one house
6,age_of_respondent,The age of the interviewee
7,gender_of_respondent,"Gender of interviewee: Male, Female"
8,relationship_with_head,The interviewee’s relationship with the head o...
9,marital_status,The martial status of the interviewee: Married...


In [11]:
# Encoding categorical variables
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['location_type'] = label_encoder.fit_transform(data['location_type'])
data['cellphone_access'] = label_encoder.fit_transform(data['cellphone_access'])
data['gender_of_respondent'] = label_encoder.fit_transform(data['gender_of_respondent'])
data['relationship_with_head'] = label_encoder.fit_transform(data['relationship_with_head'])
data['marital_status'] = label_encoder.fit_transform(data['marital_status'])
data['education_level'] = label_encoder.fit_transform(data['education_level'])
data['job_type'] = label_encoder.fit_transform(data['job_type'])

# Handling missing values
data.fillna(method='ffill', inplace=True)

# Splitting the data into features and target variable
X = data.drop(['bank_account', 'uniqueid', 'country', 'year'], axis=1)
y = data['bank_account']

# Splitting the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\This  PC\AppData\Local\Temp\ipykernel_6176\2784086994.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [16]:
# Import Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier

# Initialize the model
rf_classifier = RandomForestClassifier(random_state=42)

# Fit the model
rf_classifier.fit(X_train, y_train)

# Predictions
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = classification_report(y_test, y_pred)
print(accuracy)


              precision    recall  f1-score   support

          No       0.90      0.94      0.92      4063
         Yes       0.48      0.37      0.42       642

    accuracy                           0.86      4705
   macro avg       0.69      0.65      0.67      4705
weighted avg       0.85      0.86      0.85      4705



In [13]:
# Import cross-validation
from sklearn.model_selection import cross_val_score
import numpy as np

# Perform cross-validation
cv_scores = cross_val_score(rf_classifier, X, y, cv=5)
print("Cross Validation Scores:", cv_scores)
print("Mean CV Accuracy:", np.mean(cv_scores))


Cross Validation Scores: [0.76684378 0.83230606 0.83889479 0.82231668 0.81335034]
Mean CV Accuracy: 0.8147423315766265


In [14]:
# Import cross-validation
from sklearn.model_selection import cross_val_score, KFold

# Define the number of folds
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(rf_classifier, X, y, cv=k_fold)

# Print cross-validation scores
print("Cross Validation Scores:", cv_scores)
print("Mean CV Accuracy:", np.mean(cv_scores))


Cross Validation Scores: [0.83549416 0.84080765 0.83528162 0.8282678  0.83694728]
Mean CV Accuracy: 0.8353597002031418


Explanation:

KFold: This function splits the dataset into k folds. We specify the number of splits (n_splits) and set shuffle=True to shuffle the data before splitting.

cross_val_score: This function performs cross-validation by splitting the data using the specified cross-validator (k_fold in this case), fitting the model on each training fold, and evaluating it on the corresponding validation fold. It returns an array of scores for each fold.

cv_scores: This array contains the performance scores (e.g., accuracy) for each fold.

np.mean(cv_scores): We calculate the mean of the cross-validation scores to obtain the average performance of the model across all folds.

By using k-fold cross-validation, we ensure that the model's performance is evaluated on multiple subsets of the data, which helps in detecting and mitigating overfitting. It provides a more robust estimate of the model's generalization ability compared to a single train-test split.

In [17]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)


In [15]:
# Import Streamlit
import streamlit as st

# Create the Streamlit app
def main():
    st.title("Bank Account Prediction")
    st.sidebar.title("Bank Account Prediction")
    st.sidebar.markdown("Predict whether an individual is likely to have a bank account.")

    # Display the dataset
    if st.sidebar.checkbox("Show Dataset"):
        st.write(data)

    # Prediction
    st.sidebar.subheader("Make a Prediction")
    features = []
    for feature in X.columns:
        value = st.sidebar.selectbox(f"Select {feature}", data[feature].unique())
        features.append(value)
    
    prediction = rf_classifier.predict([features])
    if prediction[0] == 1:
        st.write("The individual is likely to have a bank account.")
    else:
        st.write("The individual is unlikely to have a bank account.")

if __name__ == '__main__':
    main()


2024-05-16 19:57:54.297 
  command:

    streamlit run c:\Users\This  PC\Documents\Financial\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-05-16 19:57:54.300 Session state does not function when running a script without `streamlit run`
c:\Users\This  PC\Documents\Financial\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [18]:
X.columns

Index(['location_type', 'cellphone_access', 'household_size',
       'age_of_respondent', 'gender_of_respondent', 'relationship_with_head',
       'marital_status', 'education_level', 'job_type'],
      dtype='object')

In [19]:
test_set = X_test.to_csv("test.csv", index=False)